In [1]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle
from sklearn.utils import shuffle
from importlib import reload
from langdetect import detect
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from dateutil.parser import parse
import spacy
from time import time

sys.path.append('../src')

from commons import elastic
from text_processing import text_normalizer
text_normalizer = reload(text_normalizer)
from text_processing import duplicate_finder
from utilities import excel_writer, excel_reader, utils
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)

stemmer = SnowballStemmer("english")
lmtzr = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver([])
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

# Load dataset

In [3]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/sample_reaper.xlsx")

Read file ../tmp/sample_reaper.xlsx: 0.03s
Processed file ../tmp/sample_reaper.xlsx: 0.02s


In [4]:
print("Number of articles: ", len(df))
print("Available columns: ", df.columns)
df.head()

Number of articles:  291
Available columns:  Index(['abstract', 'affiliation', 'authors', 'journal', 'keywords',
       'identificators', 'publisher', 'title', 'url', 'year'],
      dtype='object')


,abstract,affiliation,authors,journal,keywords,identificators,publisher,title,url,year
0,​The International Fund for Agricultural Devel...,,,,Climate action;The R4 Rural Resilience Initiat...,,,2018- Using insurance to protect farmers again...,https://docs.wfp.org/api/documents/WFP-0000073...,2018
1,<b>Problem statement:</b> Contract Farming (CF...,,Norsida Man;Nolila M. Nawi,,,,,The Practices of Contract Farming Among Fresh ...,http://agris.fao.org/agris-search/search.do?re...,2010
2,A farm business advisor for the Malawi Ministr...,,CTA;Technical Centre for Agricultural and Rura...,,,,,Competing in the open market,http://agris.fao.org/agris-search/search.do?re...,2002
3,A number of the commercially available maize c...,,"Mphundi, Patricia Makuzana",,"Dissertation (M.Sc.Agric. (Soil, Crop and Clim...",,University of the Free State,Factors affecting maize (Zea mays L.) sensitiv...,http://hdl.handle.net/11660/1701,2009
4,A recently commercialized natural bio-stimulan...,,"Hüster, Thomas",,"Thesis (Ph.D. (Soil, Crop and Climate Sciences...",,University of the Free State,Growth and yield response of selected crops to...,http://hdl.handle.net/11660/986,2011


# Data cleaning and processing

In [5]:
_duplicate_finder = duplicate_finder.DuplicateFinder("../tmp/duplicate_reaper_info")
df = _duplicate_finder.remove_duplicates_in_one_df_by_title(df, [])
print("After deduplication: ", len(df))

0.04603099822998047
0.9293808937072754
Year 2018: 29 items
Year 2010: 16 items
Year 2002: 7 items
Year 2009: 18 items
Year 2011: 24 items
Year 2015: 31 items
Year 2012: 16 items
Year 2016: 22 items
Year 2019: 7 items
Year 2014: 20 items
Year 2008: 11 items
Year 2005: 2 items
Year 2004: 9 items
Year 2006: 11 items
Year 2017: 26 items
Year 2003: 5 items
Year 2013: 19 items
Year 2007: 5 items
Year 2001: 2 items
Checked all articles
Saving...
Saved to ../tmp/duplicate_reaper_info/duplicate_in_one_dataset_1628588007.xlsx
Merged file
Filtered dataset
After deduplication:  285


In [6]:
from text_processing import advanced_text_normalization
advanced_text_normalization = reload(advanced_text_normalization)
from text_processing import all_column_filler
all_column_filler = reload(all_column_filler)

_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(df, [], _abbreviations_resolver,settings_json = {"columns":[
    {"column_filler_class":"AdvancedTextNormalizer","full_normalization":True}
]})

Started processing  {'column_filler_class': 'AdvancedTextNormalizer', 'full_normalization': True}
Processed 0 articles
Processed 284 articles
Processed 0 articles
Processed 284 articles
Processed 0 articles
Processed 284 articles
the agriculturethe
the thestudy
with withagricultural
were weredata
agriculturethe ### agriculture the ### agriculturethe ### the
withagricultural ### with agricultural ### withagricultural ### with
weredata ### were data ### weredata ### were
Processed for 8.113651037216187s


In [26]:
df["raw_affiliation"] = df["affiliation"]
df["raw_authors"] = df["authors"]

In [27]:
excel_writer.ExcelWriter().save_df_in_excel(df, "../tmp/sample_reaper_cleaned.xlsx")

Saving...
Saved to ../tmp/sample_reaper_cleaned.xlsx


# Enriching the dataset with different extraction/classification parts

In [2]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/sample_reaper_cleaned.xlsx")

Read file ../tmp/sample_reaper_cleaned.xlsx: 0.04s
Processed file ../tmp/sample_reaper_cleaned.xlsx: 0.02s


In [3]:
from text_processing import search_engine_insensitive_to_spelling
search_engine_insensitive_to_spelling = reload(search_engine_insensitive_to_spelling)

folder_with_index = "../model/index_sample_reaper"
search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
    load_abbreviations = True)
if os.path.exists(folder_with_index):
    search_engine_inverted_index.load_model(folder_with_index)
else:
    search_engine_inverted_index.create_inverted_index(df)
    search_engine_inverted_index.save_model(folder_with_index)

In [4]:
from text_processing import all_column_filler
all_column_filler = reload(all_column_filler)
from text_processing import author_normalizer
author_normalizer = reload(author_normalizer)

_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(df, search_engine_inverted_index, _abbreviations_resolver,
                                            settings_json = {"columns":[
    {"column_filler_class":"KeywordNormalizer"},
    {"column_filler_class":"JournalNormalizer"},
    {"column_filler_class":"AuthorAndAuthorAffiliationExtractor", "author_mapping": {"raw_authors": "authors"},
     "affiliations_mapping": {"raw_affiliation": "affiliation"}},
    {"column_filler_class":"GeoNameFinder"},
    {"column_filler_class":"CropsSearch", "file_dictionary":"../data/map_plant_products.xlsx", 
     "column_name":"plant_products_search"},
    {"column_filler_class":"CropsSearch", "file_dictionary":"../data/map_animal_products.xlsx",
     "column_name":"animal_products_search"},
    {"column_filler_class":"CropsSearch", "column_name": "animals_found", "keep_hierarchy": False,
     "file_dictionary":"../data/map_animals.xlsx"},
    {"column_filler_class":"PopulationTagsFinder"},
    {"column_filler_class":"ColumnFiller", "column_name": "gender_age_population_tags", 
     "keep_hierarchy": False, "column_dictionary":"../data/population_tags.xlsx"},
    {"column_filler_class":"InterventionsSearchForLabeling", "file":"../tmp/interventions.xlsx"},
    {"column_filler_class":"MeasurementsLabeler", "model_folder": "../model/bert_measurements_model",
     "folder_with_measurements":"../tmp/sample_reaper_measurements",
     "mappings_for_measurements":{
        "interventions_found_raw":"measurements_for_interventions",\
        "plant_products_search_details":"measurements_for_crops",\
        "animal_products_search_details":"measurements_for_crops",\
        "animals_found_details": "measurements_for_animals"
    }},
    {"column_filler_class": "ProgramExtractor", "column_name": "programs_found", "model_type": "model",
     "model_folder": "../model/programs_extraction_model_2619", "columns_to_process": ["title", "abstract"]},
    {"column_filler_class": "StudyTypeLabeller", "folder": "../model/study_type_multi",
     "meta_folder": "../model/study_type_multi_meta_agg",
     "scibert_model_folder": "../model/scibert_scivocab_uncased"},
    {"column_filler_class": "OutcomesFinder", "model_folder": "../model/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7"}
]})

Started processing  {'column_filler_class': 'KeywordNormalizer'}
Processed for 1.087775468826294s
Started processing  {'column_filler_class': 'JournalNormalizer'}
Processed for 0.0032663345336914062s
Started processing  {'column_filler_class': 'AuthorAndAuthorAffiliationExtractor', 'author_mapping': {'raw_authors': 'authors'}, 'affiliations_mapping': {'raw_affiliation': 'affiliation'}}
Read file ../data/provinces.xlsx: 1.87s
Processed file ../data/provinces.xlsx: 1.56s
Read file ../data/districts.xlsx: 1.54s
Processed file ../data/districts.xlsx: 1.70s
Read file ../data/map_affiliations.xlsx: 0.01s
Processed file ../data/map_affiliations.xlsx: 0.00s
Read file ../data/map_clipped_words.xlsx: 0.00s
Processed file ../data/map_clipped_words.xlsx: 0.00s
Length of map of affiliations: 29
Count of unique normalized affiliations: 29
started affiliation processing
Process 0 articles
Process 284 articles
done in 3.045s.
Processed 0 articles
Processed 284 articles
Processed for 15.915337324142456

Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '../model/bert_measurements_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8d790dc470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Processed for 28.619194746017456s


In [5]:
from interventions_labeling_lib.usaid_intervention_labels import USAIDInterventionLabels
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)

_intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
    "../model/bert_usaid_interventions_multi_label_3",
    [0 for i in range(11)], label_column="label",
    multilabel=True
)
ner_model = spacy.load("../model/usaid_entity-recognition-model-more-3012-169_iter")

found_data_all = []
idx = 0
idx2doc = {}
for i in range(len(df)):
    for sentence in nltk.sent_tokenize(re.sub("\s+", " ", df["abstract"].values[i])):
        found_data = []
        results = set()
        for res in ner_model(sentence).ents:
            intervention = _abbreviations_resolver.replace_abbreviations(
                text=text_normalizer.normalize_sentence(res.text)).strip()
            intervnetion_words_set = set()
            intervnetion_words = []
            for w in intervention.split(" "):
                if w not in intervnetion_words_set:
                    intervnetion_words_set.add(w)
                    intervnetion_words.append(w)
            intervention = " ".join(intervnetion_words).strip()
            if intervention:
                found_data.append((intervention, intervention, sentence, res.text))
                idx2doc[idx] = i
                idx += 1
        found_data_all.extend(found_data)
df_to_predict = pd.DataFrame(found_data_all, columns=["Narrow concept", "Broad concepts", "Sentence", "Raw intervention"])
res_prob, res_label, _ = _intervention_labeler.predict_for_df(df_to_predict)

df["extracted_interventions"] = ""
df["extracted_interventions_probs"] = ""
for i in range(len(df)):
    df["extracted_interventions"].values[i] = []
    df["extracted_interventions_probs"].values[i] = []

for i in range(len(found_data_all)):
    idx = idx2doc[i]
    for j in range(len(res_label[i])):
        if res_label[i][j] == 1:
            found_label = USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[j+1]
            found_prob = res_prob[i][j]
            df["extracted_interventions"].values[idx].append(found_label + "/" + found_data_all[i][0])
            df["extracted_interventions_probs"].values[idx].append((found_label + "/" + found_data_all[i][0], found_prob))

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '../model/bert_usaid_interventions_multi_label_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8d101d1588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '../model/bert_usaid_interventions_multi_label_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8d101d1588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 1700


...

INFO:tensorflow:Restoring parameters from ../model/bert_usaid_interventions_multi_label_3/model.ckpt-1108


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [7]:
excel_writer.ExcelWriter().save_df_in_excel(df, "../tmp/sample_reaper_enriched.xlsx")

Saving...
Saved to ../tmp/sample_reaper_enriched.xlsx
